In [1]:
import pandas as pd
import geopandas as gpd
user = 'aolsen'

/Users/aolsen/opt/anaconda3/envs/geo_env_2/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_35726/215051122.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shap

In [2]:
bayareafips_full = {'06001': 'Alameda', '06013': 'Contra Costa', '06041': 'Marin', '06055': 'Napa',
                    '06075': 'San Francisco', '06081': 'San Mateo', '06085': 'Santa Clara',
                    '06097': 'Sonoma', '06095': 'Solano'}

# Load data

In [3]:
# tazs
ZONES_PATH = f'/Users/{user}/Box/Modeling and Surveys/Urban Modeling/Spatial/Zones/TAZ1454/zones1454.shp'

zones = gpd.read_file(ZONES_PATH).to_crs('EPSG:26910')
zones['geom_pt'] = zones.representative_point()

## Observed data / Shimon

In [4]:
taz_2020 = pd.read_csv(
    'https://raw.githubusercontent.com/BayAreaMetro/petrale/main/applications/travel_model_lu_inputs/2020/TAZ1454_2020_long.csv')
taz_2023 = pd.read_csv(
    'https://raw.githubusercontent.com/BayAreaMetro/petrale/main/applications/travel_model_lu_inputs/2023/TAZ1454_2023_long.csv')

In [5]:
# taz_2020['SD'] = taz_2020.ZONE.map(zones.set_index('taz1454').superdistr)
# taz_2023['SD'] = taz_2023.ZONE.map(zones.set_index('taz1454').superdistr)

## PBA 50 data - taz

In [6]:
tazdata_2010 = pd.read_csv(f'/Users/{user}/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182_taz_summaries_2010.csv')
tazdata_2010['superdistrict'] = tazdata_2010.SD

In [7]:
tazdata_2020 = pd.read_csv(f'/Users/{user}/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/run182_taz_summaries_2020.csv')
tazdata_2020['superdistrict'] = tazdata_2020.SD

In [8]:
superdistricts = zones.dissolve('superdistr')

In [9]:
# get mapping - surely we have this somewhere in a text file

superdistrict_to_county_map = zones.groupby(
    ['superdistr', 'fipsstco']).size().reset_index(1).fipsstco.map(bayareafips_full)

## Census 2010

In [10]:
census2010_block = gpd.read_file(f'/Users/{user}/Box/Modeling and Surveys/Census/censusblocks2010_w_dat.gpkg')
census2010_block['geom_pt'] = census2010_block.representative_point()

In [11]:
# assign containing zone to census block
bayarea_blocks_2010_x_zones = gpd.sjoin(
    census2010_block[['GEOID10','geom_pt']].set_geometry('geom_pt'), zones)


In [12]:
census2010_block['zone_id'] = census2010_block.GEOID10.map(bayarea_blocks_2010_x_zones.groupby(['GEOID10']).taz1454.first())
census2010_block['superdistrict'] = census2010_block.GEOID10.map(bayarea_blocks_2010_x_zones.groupby(['GEOID10']).superdistr.first())

## Census 2020

In [13]:
census2020_block = gpd.read_file(f'/Users/{user}/Box/Modeling and Surveys/Census/censusblocks2020_w_dat.gpkg')

In [14]:
#census_2020_data = pd.read_excel('/Users/{user}/Downloads/census_2020_dhc_block_vars.xlsx')

In [15]:
census2020_block['superdistrict'] = census2020_block.zone_id.map(zones.set_index('taz1454').superdistr)

In [16]:
census2010_block.groupby(['superdistrict'])[['population','households']].sum().head()

,population,households
superdistrict,,
1.0,139649,75771
2.0,206581,101971
3.0,324381,118333
4.0,134624,49736
5.0,290434,98382


# Comparisons

In [23]:
# 2020 observations

valid_2020 = pd.concat([
    tazdata_2010.groupby(['superdistrict']).TOTHH.sum(),
    tazdata_2020.groupby(['superdistrict']).TOTHH.sum(),
    census2010_block.groupby(['superdistrict']).households.sum(),
    census2020_block.groupby(['superdistrict']).households.sum(),
    taz_2020.query('Variable=="TOTHH" & Year==2020').groupby(
        ['DISTRICT']).Value.sum(),
    taz_2023.query('Variable=="TOTHH" & Year==2023').groupby(
        ['DISTRICT']).Value.sum()
],  # check, then double check the ordering matches the above list
    keys=['baseyear2010_run182', 
          'modeled2020_run182',
          'observed_census2010',
          'observed_census2020',
          'observed_acs2020',
          'estim_acs2023'],
    names=['what'])
valid_2020 = valid_2020.unstack([0])  # .reset_index(name='value')

In [24]:
# store comparison expressions in a dict for better readability

comparisons = {
    # model growth relative to *census* 2010
    'ch_182_minus_c2010': 'modeled2020_run182 - observed_census2010',
    # model growth relative to *model* 2010
    'ch_182_minus_182': 'modeled2020_run182 - baseyear2010_run182',

    # growth 2010-2020 per census data
    'ch_c2020_minus_c2010': 'observed_census2020 - observed_census2010',

    # growth 2010-2023 per census data
    'ch_acs2023_minus_c2010': 'estim_acs2023 - observed_census2010',

    # ratio of model growth to observed growth
    'rat_modelgr_vs_obsgr': 'ch_182_minus_182 / ch_c2020_minus_c2010'}

In [25]:
# Mean average percentage error

# manual approach
mean_MAPE = ((valid_2020.modeled2020_run182 - valid_2020.observed_census2020) / \
    valid_2020.observed_census2020)
print(mean_MAPE.mean())

# vectorized approach
model_var = 'modeled2020_run182'
reference_var = 'observed_census2020'
print(valid_2020[[reference_var,model_var]].pct_change(axis=1)[model_var].mean())

-0.0017301432654833679
-0.0017301432654833586


In [26]:
# turn into df
valid_2020_df = valid_2020.reset_index()
valid_2020_df['county'] = valid_2020_df.superdistrict.map(
    superdistrict_to_county_map)
valid_2020_df['geometry'] = valid_2020_df.superdistrict.map(
    superdistricts.geometry)
valid_2020_df = valid_2020_df.set_geometry('geometry')

In [27]:
# loop through comparison expresions; assign to dataframe
for var, expr in comparisons.items():
    print(var, expr)
    valid_2020_df[var] = valid_2020_df.eval(expr)
    valid_2020_df[var]

ch_182_minus_c2010 modeled2020_run182 - observed_census2010
ch_182_minus_182 modeled2020_run182 - baseyear2010_run182
ch_c2020_minus_c2010 observed_census2020 - observed_census2010
ch_acs2023_minus_c2010 estim_acs2023 - observed_census2010
rat_modelgr_vs_obsgr ch_182_minus_182 / ch_c2020_minus_c2010


# Output
Write out to tableau

In [30]:
out_path = f'/Users/{user}/Downloads/baus_2020_validation.geojson'

valid_2020_df.to_file(out_path,driver='GeoJSON')